In [ ]:
"""
Hierarchical Neural Rayleigh-Ritz Eigensolver
---------------------------------------------
Author: <you>
Version: 2.1 (fixed + includes small/medium tests)
"""

import numpy as np
import torch
import torch.nn as nn
from scipy.linalg import eigh
from scipy.sparse.linalg import eigsh
from scipy.sparse import csr_matrix


# ===============================================================
# MATRIX HIERARCHY
# ===============================================================

def build_hierarchy(K_full, M_full, levels=None, method='uniform'):
    """
    Build matrix hierarchy by downsampling.
    """
    if levels is None:
        levels = {0: 2**5, 1: 2**10, 2: 2**15, 3: 2**20}

    n_full = K_full.shape[0]
    K_hierarchy, M_hierarchy, idx_hierarchy = {}, {}, {}

    for level, size in sorted(levels.items()):
        size = int(size)
        if size >= n_full:
            idx = np.arange(n_full)
        else:
            if method == 'uniform':
                idx = np.linspace(0, n_full - 1, num=size)
                idx = np.unique(np.round(idx).astype(int))
                if idx.size < size:
                    remaining = set(range(n_full)) - set(idx)
                    extra = np.array(sorted(list(remaining)))[: (size - idx.size)]
                    idx = np.sort(np.concatenate([idx, extra]))
            elif method == 'random':
                np.random.seed(42 + level)
                idx = np.sort(np.random.choice(n_full, size, replace=False))
            elif method == 'importance':
                idx = leverage_score_sampling(K_full, size, seed=42 + level)
            else:
                raise ValueError(f"Unknown method: {method}")

        K_hierarchy[level] = K_full[np.ix_(idx, idx)]
        M_hierarchy[level] = M_full[np.ix_(idx, idx)]
        idx_hierarchy[level] = idx

        print(f"Built level {level}: {size} DOFs")

    return K_hierarchy, M_hierarchy, idx_hierarchy


def leverage_score_sampling(K, target_size, seed=0):
    """Approximate leverage-score sampling."""
    n = K.shape[0]
    if n > 10000:
        scores = np.abs(np.diag(K)).astype(float)
    else:
        scores = np.sum(np.abs(K) ** 2, axis=1).astype(float)
    scores = 0.9 * (scores / np.sum(scores)) + 0.1 / n
    np.random.seed(seed)
    idx = np.sort(np.random.choice(n, target_size, replace=False, p=scores / scores.sum()))
    return idx


# ===============================================================
# RAYLEIGH–RITZ REFINEMENT
# ===============================================================

def simple_rayleigh_ritz_refinement(λ_init, u_init, K, M, n_iter=5):
    """Simple Rayleigh–Ritz refinement."""
    λ_refined, u_refined = λ_init.copy(), u_init.copy()
    print(f"  Simple Rayleigh–Ritz refinement: {n_iter} iterations")

    for it in range(n_iter):
        for i in range(len(λ_refined)):
            u = u_refined[:, i]
            Ku, Mu = K @ u, M @ u
            λ_refined[i] = (u.T @ Ku) / (u.T @ Mu)
            for j in range(i):
                overlap = u.T @ M @ u_refined[:, j]
                u -= overlap * u_refined[:, j]
            u_refined[:, i] = u / np.sqrt(u.T @ M @ u)

            if it == n_iter - 1:
                res = np.linalg.norm(K @ u - λ_refined[i] * M @ u)
                print(f"    Eigenpair {i}: residual={res:.2e}, λ={λ_refined[i]:.6f}")

    return λ_refined, u_refined


# ===============================================================
# UPSCALER NETWORK
# ===============================================================

class HierarchicalUpscaler(nn.Module):
    """Neural upscaler network."""

    def __init__(self, n_coarse, n_fine, hidden_factor=4):
        super().__init__()
        hidden = min(n_coarse * hidden_factor, max(32, n_fine // 2))
        self.net = nn.Sequential(
            nn.Linear(n_coarse, hidden),
            nn.Tanh(),
            nn.Linear(hidden, hidden),
            nn.Tanh(),
            nn.Linear(hidden, n_fine)
        )

    def forward(self, u_coarse):
        was_1d = False
        if u_coarse.dim() == 1:
            u_coarse = u_coarse.unsqueeze(0)
            was_1d = True
        out = self.net(u_coarse)
        return out.squeeze(0) if was_1d else out


# ===============================================================
# LOSS & TRAINING
# ===============================================================

def compute_loss(u_fine, λ, K_fine, M_fine, u_previous=None, weights=None):
    if weights is None:
        weights = {'residual': 1.0, 'normalization': 10.0,
                   'orthogonality': 5.0, 'smoothness': 0.01}

    Ku = K_fine @ u_fine
    Mu = M_fine @ u_fine
    residual = Ku - λ * Mu

    losses = {
        'residual': torch.mean(residual ** 2),
        'normalization': (u_fine.T @ Mu - 1.0) ** 2,
        'orthogonality': torch.zeros((), device=u_fine.device),
        'smoothness': torch.mean((u_fine[1:] - u_fine[:-1]) ** 2) if u_fine.numel() > 1 else 0
    }

    if u_previous is not None:
        for u_prev in u_previous:
            if not isinstance(u_prev, torch.Tensor):
                u_prev = torch.tensor(u_prev, dtype=u_fine.dtype, device=u_fine.device)
            overlap = (u_fine.T @ (M_fine @ u_prev))
            losses['orthogonality'] += overlap ** 2

    total = sum(weights[k] * losses[k] for k in losses)
    return total, losses


def adaptive_weights(epoch):
    return {
        'residual': 1.0,
        'normalization': 10.0 * np.exp(-epoch / 100),
        'orthogonality': 5.0,
        'smoothness': 0.01
    }


def train_upscaler(level_from, level_to,
                   λ_coarse, u_coarse,
                   K_fine, M_fine,
                   n_eigenpairs=10,
                   n_epochs=1000,
                   device='cuda'):
    n_coarse, n_fine = u_coarse.shape[0], K_fine.shape[0]
    print(f"Training upscaler: {n_coarse} → {n_fine} (level {level_from} → {level_to})")

    λ_fine = np.zeros(n_eigenpairs)
    u_fine_all = []

    K_t = torch.tensor(K_fine, dtype=torch.float32, device=device)
    M_t = torch.tensor(M_fine, dtype=torch.float32, device=device)

    for i in range(n_eigenpairs):
        print(f"  Eigenpair {i + 1}/{n_eigenpairs}")
        model = HierarchicalUpscaler(n_coarse, n_fine).to(device)
        lam = nn.Parameter(torch.tensor(float(λ_coarse[i]), dtype=torch.float32, device=device))
        optimizer = torch.optim.Adam(list(model.parameters()) + [lam], lr=1e-3)

        u_c = torch.tensor(u_coarse[:, i], dtype=torch.float32, device=device)
        best_loss = float('inf')
        patience_counter = 0

        for epoch in range(n_epochs):
            u_f = model(u_c)
            weights = adaptive_weights(epoch)
            loss, _ = compute_loss(u_f, lam, K_t, M_t, u_previous=u_fine_all, weights=weights)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if epoch % 100 == 0:
                print(f"    Epoch {epoch}: loss={loss.item():.2e}, λ={lam.item():.6f}")

            if loss.item() < best_loss - 1e-12:
                best_loss, patience_counter = loss.item(), 0
            else:
                patience_counter += 1
                if patience_counter > 200:
                    print(f"    Early stopping at epoch {epoch}")
                    break

        λ_fine[i] = lam.detach().cpu().item()
        u_fine_all.append(u_f.detach())

    u_fine_matrix = torch.stack(u_fine_all, dim=1).cpu().numpy()
    return λ_fine, u_fine_matrix


# ===============================================================
# PIPELINE
# ===============================================================

def hierarchical_eigensolve(K_full, M_full, n_eigenpairs=10, levels=None, method='uniform', device='cuda'):
    print("Building matrix hierarchy...")
    K_hierarchy, M_hierarchy, _ = build_hierarchy(K_full, M_full, levels, method)
    level_keys = sorted(K_hierarchy.keys())

    K_coarse, M_coarse = K_hierarchy[level_keys[0]], M_hierarchy[level_keys[0]]
    print(f"Coarsest level size: {K_coarse.shape}")

    if K_coarse.shape[0] <= 2000:
        λ_current, u_current = eigh(K_coarse, M_coarse)
    else:
        λ_current, u_current = eigsh(csr_matrix(K_coarse), k=n_eigenpairs, M=csr_matrix(M_coarse), which='SM')

    λ_current = λ_current[:n_eigenpairs]
    u_current = u_current[:, :n_eigenpairs]
    print(f"Coarsest eigenvalues: {λ_current}")

    for i in range(1, len(level_keys)):
        l_from, l_to = level_keys[i - 1], level_keys[i]
        print(f"\nRefinement step {i}: level {l_from} → {l_to}")
        Kf, Mf = K_hierarchy[l_to], M_hierarchy[l_to]
        λ_current, u_current = train_upscaler(l_from, l_to, λ_current, u_current, Kf, Mf,
                                              n_eigenpairs=n_eigenpairs, device=device)
        λ_current, u_current = simple_rayleigh_ritz_refinement(λ_current, u_current, Kf, Mf, n_iter=3)

    return λ_current, u_current


# ===============================================================
# TEST UTILITIES
# ===============================================================

def generate_test_matrices(n, matrix_type='laplacian'):
    if matrix_type == 'laplacian':
        K = np.diag(2 * np.ones(n)) - np.diag(np.ones(n - 1), 1) - np.diag(np.ones(n - 1), -1)
        M = np.eye(n)
    elif matrix_type == 'tridiagonal':
        main_diag = 2 + 0.1 * np.arange(n)
        K = np.diag(main_diag) - np.diag(np.ones(n - 1), 1) - np.diag(np.ones(n - 1), -1)
        M = np.diag(1 + 0.01 * np.arange(n))
    else:
        raise ValueError("Unknown matrix type.")
    return K, M


def verify_eigenpairs(λ, u, K, M, name=""):
    print(f"\n{name} verification")
    print("-" * 40)
    for i in range(len(λ)):
        res = np.linalg.norm(K @ u[:, i] - λ[i] * M @ u[:, i])
        print(f"  λ[{i}] = {λ[i]:.6f}, residual = {res:.2e}")
    U = u[:, :len(λ)]
    off_diag = np.abs(U.T @ M @ U - np.eye(len(λ)))
    print(f"  Max off-diagonal: {np.max(off_diag):.2e}")


# ===============================================================
# TEST RUNNERS
# ===============================================================

def run_quick_test():
    print("\n===== QUICK TEST (n=128) =====")
    K, M = generate_test_matrices(128)
    levels = {0: 16, 1: 128}
    λ, u = hierarchical_eigensolve(K, M, n_eigenpairs=3, levels=levels)
    verify_eigenpairs(λ, u, K, M, "Quick test")
    print("✓ Quick test done.")


def run_small_test():
    print("\n===== SMALL TEST (n=512) =====")
    K, M = generate_test_matrices(512)
    levels = {0: 32, 1: 128, 2: 512}
    λ, u = hierarchical_eigensolve(K, M, n_eigenpairs=5, levels=levels)
    verify_eigenpairs(λ, u, K, M, "Small test")
    print("✓ Small test done.")


def run_medium_test():
    print("\n===== MEDIUM TEST (n=4096, sparse) =====")
    K, M = generate_test_matrices(4096)
    levels = {0: 64, 1: 512, 2: 2048, 3: 4096}
    λ, u = hierarchical_eigensolve(K, M, n_eigenpairs=5, levels=levels)
    verify_eigenpairs(λ, u, K, M, "Medium test")
    print("✓ Medium test done.")


# ===============================================================
# MAIN
# ===============================================================

if __name__ == "__main__":
    run_quick_test()
    # Uncomment to test more levels:
    # run_small_test()
    # run_medium_test()


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from scipy.linalg import eigh, solve

# =========================
# Device Configuration
# =========================
def get_device():
    """Get the best available device (CUDA if available, else CPU)"""
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device('cpu')
        print("Using CPU")
    return device

# =========================
# Hierarchy Construction
# =========================
def build_hierarchy(K_full, M_full, levels=None, method='uniform'):
    """
    Build a hierarchy of downsampled matrices.
    """
    if levels is None:
        levels = {0: 32, 1: 256, 2: 2048, 3: K_full.shape[0]}

    n_full = K_full.shape[0]
    K_hierarchy, M_hierarchy, idx_hierarchy = {}, {}, {}

    for level, size in sorted(levels.items()):
        if size > n_full:
            size = n_full

        if method == 'uniform':
            idx = np.linspace(0, n_full - 1, size, dtype=int)
        elif method == 'random':
            np.random.seed(42 + level)
            idx = np.sort(np.random.choice(n_full, size, replace=False))
        else:
            raise ValueError(f"Unknown method: {method}")

        K_hierarchy[level] = K_full[np.ix_(idx, idx)]
        M_hierarchy[level] = M_full[np.ix_(idx, idx)]
        idx_hierarchy[level] = idx

        print(f"Built level {level}: {size} DOFs")

    return K_hierarchy, M_hierarchy, idx_hierarchy

# =========================
# Coarse → Fine Projection
# =========================
def project_coarse_to_fine(u_coarse, idx_coarse, n_fine, method='linear'):
    """
    Project coarse vector to fine mesh via simple linear mapping.
    """
    u_fine = np.zeros(n_fine, dtype=float)
    u_fine[idx_coarse] = u_coarse
    return u_fine

# =========================
# Neural Upscaler
# =========================
class HierarchicalUpscaler(nn.Module):
    def __init__(self, n_coarse, n_fine, hidden_factor=4):
        super().__init__()
        hidden = max(min(n_coarse * hidden_factor, n_fine // 2), 32)
        # Input: n_coarse, output: n_fine
        self.net = nn.Sequential(
            nn.Linear(n_coarse, hidden),
            nn.SiLU(),
            nn.Linear(hidden, hidden),
            nn.SiLU(),
            nn.Linear(hidden, n_fine)
        )

    def forward(self, u_coarse):
        """
        u_coarse: (n_coarse,) or (batch, n_coarse)
        Returns: (n_fine,) or (batch, n_fine)
        """
        if u_coarse.dim() == 1:
            return self.net(u_coarse.unsqueeze(0)).squeeze(0)
        else:
            return self.net(u_coarse)

# =========================
# Loss Function
# =========================
def compute_loss(u_fine, λ, K_fine, M_fine, u_previous=None, weights=None):
    if weights is None:
        weights = {'residual': 1.0, 'normalization': 10.0, 'orthogonality': 5.0, 'smoothness': 0.01}

    losses = {}
    Ku = K_fine @ u_fine
    Mu = M_fine @ u_fine

    # Residual: (K - λM)u ≈ 0
    losses['residual'] = torch.mean((Ku - λ * Mu) ** 2)
    
    # Normalization: u^T M u = 1
    losses['normalization'] = ((u_fine.T @ Mu) - 1.0) ** 2

    # Orthogonality to previous eigenvectors
    losses['orthogonality'] = torch.tensor(0.0, device=u_fine.device, dtype=u_fine.dtype)
    if u_previous is not None and len(u_previous) > 0:
        for u_prev in u_previous:
            overlap = (u_fine.T @ (M_fine @ u_prev))
            losses['orthogonality'] += overlap ** 2

    # Smoothness regularization
    losses['smoothness'] = torch.mean((u_fine[1:] - u_fine[:-1]) ** 2)
    
    total = sum(weights[k] * losses[k] for k in losses)
    return total, losses

def adaptive_weights(epoch, losses_history=None):
    return {
        'residual': 1.0,
        'normalization': 10.0 * np.exp(-epoch / 100),
        'orthogonality': 5.0,
        'smoothness': 0.01
    }

# =========================
# Upscaler Training
# =========================
def train_upscaler(level_from, level_to, λ_coarse, u_coarse, K_fine, M_fine, idx_coarse, 
                   n_eigenpairs=10, n_epochs=1000, lr=1e-3, device=None):
    if device is None:
        device = get_device()
    
    n_coarse = u_coarse.shape[0]
    n_fine = K_fine.shape[0]

    print(f"Training upscaler: {n_coarse} → {n_fine} (level {level_from} → {level_to})")

    λ_fine = torch.zeros(n_eigenpairs)
    u_fine_all = []

    # Move matrices to GPU
    K_t = torch.tensor(K_fine, dtype=torch.float32, device=device)
    M_t = torch.tensor(M_fine, dtype=torch.float32, device=device)

    for i in range(n_eigenpairs):
        print(f"  Eigenpair {i+1}/{n_eigenpairs}")

        # Initialize eigenvalue from coarse solution
        λ_init = λ_coarse[i]
        
        # Convert coarse eigenvector to tensor and move to GPU
        u_c = torch.tensor(u_coarse[:, i], dtype=torch.float32, device=device)
        
        # Initialize eigenvalue as trainable parameter on GPU
        λ = torch.tensor(λ_init, dtype=torch.float32, device=device, requires_grad=True)

        # Create and move upscaler model to GPU
        model = HierarchicalUpscaler(n_coarse, n_fine).to(device)
        optimizer = torch.optim.Adam(list(model.parameters()) + [λ], lr=lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=50, factor=0.5)

        best_loss = float('inf')
        patience_counter = 0

        for epoch in range(n_epochs):
            # Upscale coarse eigenvector to fine mesh
            u_f = model(u_c)
            
            # Compute loss with adaptive weights
            weights = adaptive_weights(epoch)
            loss, loss_dict = compute_loss(u_f, λ, K_t, M_t, u_previous=u_fine_all, weights=weights)

            # Optimization step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step(loss)

            # Logging
            if epoch % 100 == 0:
                print(f"    Epoch {epoch}: Loss={loss.item():.2e}, λ={λ.item():.6f}, "
                      f"residual={loss_dict['residual'].item():.2e}")

            # Early stopping
            if loss.item() < best_loss:
                best_loss = loss.item()
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter > 200:
                    print(f"    Early stopping at epoch {epoch}")
                    break

        # Store results (move back to CPU for storage)
        λ_fine[i] = λ.detach().cpu()
        u_fine_all.append(u_f.detach())  # Keep on GPU for next iteration

    # Convert to numpy (move to CPU)
    u_fine_matrix = torch.stack(u_fine_all, dim=1).cpu().numpy()
    return λ_fine.numpy(), u_fine_matrix

# =========================
# Simple Rayleigh–Ritz Refinement
# =========================
def simple_rayleigh_ritz_refinement(λ_init, u_init, K, M, n_iter=3):
    """
    Refine eigenpairs using Rayleigh quotient iteration and Gram-Schmidt orthogonalization.
    """
    λ_refined = λ_init.copy()
    u_refined = u_init.copy()

    for it in range(n_iter):
        for i in range(len(λ_init)):
            u = u_refined[:, i]
            Ku = K @ u
            Mu = M @ u
            
            # Update eigenvalue using Rayleigh quotient
            λ_refined[i] = (u.T @ Ku) / (u.T @ Mu)

            # Orthogonalize against previous eigenvectors
            for j in range(i):
                overlap = u.T @ M @ u_refined[:, j]
                u -= overlap * u_refined[:, j]

            # Normalize
            norm = np.sqrt(u.T @ M @ u)
            if norm > 1e-10:
                u_refined[:, i] = u / norm
            else:
                print(f"Warning: Near-zero norm at iteration {it}, eigenvector {i}")

    return λ_refined, u_refined

# =========================
# Hierarchical Eigen Solve
# =========================
def hierarchical_eigensolve(K_full, M_full, n_eigenpairs=10, method='uniform', levels=None, device=None):
    """
    Solve generalized eigenvalue problem K u = λ M u using hierarchical neural upscaling.
    """
    if device is None:
        device = get_device()
    
    # Build hierarchy of coarse meshes
    K_h, M_h, idx_h = build_hierarchy(K_full, M_full, levels=levels, method=method)
    level_keys = sorted(K_h.keys())

    # Solve coarsest level exactly (on CPU with scipy)
    print("\nSolving coarsest level...")
    K_coarse = K_h[level_keys[0]]
    M_coarse = M_h[level_keys[0]]
    λ_current, u_current = eigh(K_coarse, M_coarse)
    λ_current = λ_current[:n_eigenpairs]
    u_current = u_current[:, :n_eigenpairs]
    
    print(f"Coarsest eigenvalues: {λ_current[:min(5, n_eigenpairs)]}")

    # Progressive refinement through hierarchy
    for i in range(1, len(level_keys)):
        level_from = level_keys[i-1]
        level_to = level_keys[i]

        print(f"\n--- Refining from level {level_from} to level {level_to} ---")
        
        # Neural upscaling (on GPU)
        λ_current, u_current = train_upscaler(
            level_from, level_to,
            λ_current, u_current,
            K_h[level_to], M_h[level_to],
            idx_coarse=idx_h[level_from],
            n_eigenpairs=n_eigenpairs,
            device=device
        )

        # Post-refinement with Rayleigh-Ritz (on CPU)
        print(f"  Applying Rayleigh-Ritz refinement...")
        λ_current, u_current = simple_rayleigh_ritz_refinement(
            λ_current, u_current,
            K_h[level_to], M_h[level_to],
            n_iter=3
        )
        
        print(f"  Refined eigenvalues: {λ_current[:min(5, n_eigenpairs)]}")

    return λ_current, u_current

# =========================
# Test Matrices
# =========================
def generate_test_matrices(n, matrix_type='laplacian'):
    """
    Generate test matrix pairs (K, M) for eigenvalue problems.
    """
    if matrix_type == 'laplacian':
        K = np.diag(2 * np.ones(n)) - np.diag(np.ones(n-1), 1) - np.diag(np.ones(n-1), -1)
        M = np.eye(n)
    elif matrix_type == 'tridiagonal':
        K = np.diag(2 + 0.1 * np.arange(n)) - np.diag(np.ones(n-1), 1) - np.diag(np.ones(n-1), -1)
        M = np.diag(1 + 0.01 * np.arange(n))
    elif matrix_type == 'random_spd':
        np.random.seed(42)
        A = np.random.randn(n, n)
        B = np.random.randn(n, n)
        K = A @ A.T + n * np.eye(n)
        M = B @ B.T + n * np.eye(n)
    else:
        raise ValueError(f"Unknown type: {matrix_type}")
    return K, M

# =========================
# Verification
# =========================
def verify_eigenpairs(λ, u, K, M, name=""):
    """
    Verify quality of computed eigenpairs.
    """
    print(f"\n{name} Verification:")
    n_eig = len(λ)
    for i in range(n_eig):
        res = np.linalg.norm(K @ u[:, i] - λ[i] * M @ u[:, i])
        norm = u[:, i].T @ M @ u[:, i]
        print(f"  λ[{i}]={λ[i]:.6f}, residual={res:.2e}, norm={norm:.6f}")

# =========================
# Example Usage
# =========================
if __name__ == "__main__":
    # Set device
    device = get_device()
    
    # Generate test problem
    n = 4096
    K, M = generate_test_matrices(n, matrix_type='laplacian')
    
    # Solve using hierarchical method
    print("=" * 60)
    print("HIERARCHICAL EIGENSOLVE")
    print("=" * 60)
    
    λ_hier, u_hier = hierarchical_eigensolve(K, M, n_eigenpairs=10, device=device)
    verify_eigenpairs(λ_hier, u_hier, K, M, name="Hierarchical")
    
    # Compare with direct solver
    print("\n" + "=" * 60)
    print("DIRECT EIGENSOLVE (for comparison)")
    print("=" * 60)
    λ_direct, u_direct = eigh(K, M)
    λ_direct = λ_direct[:10]
    u_direct = u_direct[:, :10]
    verify_eigenpairs(λ_direct, u_direct, K, M, name="Direct")
    
    # Compare results
    print("\n" + "=" * 60)
    print("COMPARISON")
    print("=" * 60)
    print("Eigenvalue errors:")
    for i in range(10):
        err = abs(λ_hier[i] - λ_direct[i]) / λ_direct[i]
        print(f"  Mode {i}: λ_hier={λ_hier[i]:.6f}, λ_direct={λ_direct[i]:.6f}, rel_err={err:.2e}")
    
    # Clear GPU cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [34]:
print("\n===== BUNNY TEST (n = 2503) =====")
from Mesh import Mesh

m = Mesh('bunny.obj')

centroid = m.verts.mean(0)
std_max = m.verts.std(0).max()

verts_new = (m.verts - centroid)/std_max

m = Mesh(verts = verts_new, connectivity = m.connectivity)

K, M = m.computeLaplacian()
print('Computing eigen values')
eigvals, eigvecs = eigh(K,M)

#levels = {0: 256, 1: 512, 2: 1024, 3: 2503}
levels = {0: 1024, 1: 2503}
λ_hier, u_hier = hierarchical_eigensolve(K, M, n_eigenpairs=10, levels=levels, device=device)
verify_eigenpairs(λ_hier, u_hier, K, M, "Bunny test")
print("✓ Bunny test done.")


===== BUNNY TEST (n = 2503) =====
Computing eigen values
Built level 0: 1024 DOFs
Built level 1: 2503 DOFs

Solving coarsest level...
Coarsest eigenvalues: [49.23205092 50.75561199 51.81860899 54.6515191  54.8980312 ]

--- Refining from level 0 to level 1 ---
Training upscaler: 1024 → 2503 (level 0 → 1)
  Eigenpair 1/10
    Epoch 0: Loss=9.80e+00, λ=49.233051, residual=1.11e-02
    Epoch 100: Loss=2.44e-02, λ=49.285015, residual=2.40e-02
    Epoch 200: Loss=1.13e-02, λ=49.280731, residual=1.10e-02
    Epoch 300: Loss=7.64e-03, λ=49.253986, residual=7.30e-03
    Epoch 400: Loss=5.59e-03, λ=49.218933, residual=5.25e-03
    Epoch 500: Loss=4.18e-03, λ=49.183060, residual=3.82e-03
    Epoch 600: Loss=3.15e-03, λ=49.150063, residual=2.78e-03
    Epoch 700: Loss=2.39e-03, λ=49.121319, residual=2.01e-03
    Epoch 800: Loss=1.82e-03, λ=49.096901, residual=1.46e-03
    Epoch 900: Loss=1.38e-03, λ=49.076340, residual=1.06e-03
  Eigenpair 2/10
    Epoch 0: Loss=9.79e+00, λ=50.756611, residual=1.

In [35]:
λ_hier

array([47.025356, 49.5391  , 49.842953, 53.669647, 55.164696, 57.55071 ,
       57.624187, 63.459072, 63.953407, 64.781006], dtype=float32)

In [36]:
eigvals[:5]

array([3.78671343e-15, 1.60038744e-01, 4.25258130e-01, 4.38250633e-01,
       5.38463815e-01])

In [38]:
import numpy as np
import torch
import torch.nn as nn
from scipy.linalg import eigh, solve

# =========================
# Device Configuration
# =========================
def get_device():
    """Get the best available device (CUDA if available, else CPU)"""
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device('cpu')
        print("Using CPU")
    return device

# =========================
# Hierarchy Construction
# =========================
def build_hierarchy(K_full, M_full, levels=None, method='uniform', preserve_boundary=True):
    """
    Build a hierarchy of downsampled matrices.
    For meshes with null eigenvalues, this can cause spectrum shift.
    """
    if levels is None:
        levels = {0: 32, 1: 256, 2: 2048, 3: K_full.shape[0]}

    n_full = K_full.shape[0]
    K_hierarchy, M_hierarchy, idx_hierarchy = {}, {}, {}

    for level, size in sorted(levels.items()):
        if size > n_full:
            size = n_full

        if method == 'uniform':
            idx = np.linspace(0, n_full - 1, size, dtype=int)
        elif method == 'random':
            np.random.seed(42 + level)
            idx = np.sort(np.random.choice(n_full, size, replace=False))
        elif method == 'maxdist':
            # Farthest point sampling for better coverage
            idx = farthest_point_sampling(K_full, size)
        else:
            raise ValueError(f"Unknown method: {method}")

        K_hierarchy[level] = K_full[np.ix_(idx, idx)]
        M_hierarchy[level] = M_full[np.ix_(idx, idx)]
        idx_hierarchy[level] = idx

        print(f"Built level {level}: {size} DOFs")

    return K_hierarchy, M_hierarchy, idx_hierarchy

def farthest_point_sampling(K, n_samples):
    """
    Select points using farthest point sampling (better for mesh downsampling).
    Uses the mass matrix structure as a proxy for distance.
    """
    n = K.shape[0]
    if n_samples >= n:
        return np.arange(n)
    
    indices = [0]
    distances = np.full(n, np.inf)
    
    for _ in range(n_samples - 1):
        last_idx = indices[-1]
        # Update distances (simple heuristic using row norms)
        new_dist = np.abs(K[last_idx, :])
        distances = np.minimum(distances, new_dist)
        distances[indices] = -np.inf
        indices.append(np.argmax(distances))
    
    return np.sort(np.array(indices))

# =========================
# Coarse → Fine Projection
# =========================
def project_coarse_to_fine(u_coarse, idx_coarse, n_fine, method='linear'):
    """
    Project coarse vector to fine mesh via simple linear mapping.
    """
    u_fine = np.zeros(n_fine, dtype=float)
    u_fine[idx_coarse] = u_coarse
    return u_fine

# =========================
# Neural Upscaler
# =========================
class HierarchicalUpscaler(nn.Module):
    def __init__(self, n_coarse, n_fine, hidden_factor=4):
        super().__init__()
        hidden = max(min(n_coarse * hidden_factor, n_fine // 2), 32)
        # Input: n_coarse, output: n_fine
        self.net = nn.Sequential(
            nn.Linear(n_coarse, hidden),
            nn.SiLU(),
            nn.Linear(hidden, hidden),
            nn.SiLU(),
            nn.Linear(hidden, n_fine)
        )

    def forward(self, u_coarse):
        """
        u_coarse: (n_coarse,) or (batch, n_coarse)
        Returns: (n_fine,) or (batch, n_fine)
        """
        if u_coarse.dim() == 1:
            return self.net(u_coarse.unsqueeze(0)).squeeze(0)
        else:
            return self.net(u_coarse)

# =========================
# Loss Function
# =========================
def compute_loss(u_fine, λ, K_fine, M_fine, u_previous=None, weights=None):
    if weights is None:
        weights = {'residual': 1.0, 'normalization': 10.0, 'orthogonality': 5.0, 'smoothness': 0.01}

    losses = {}
    Ku = K_fine @ u_fine
    Mu = M_fine @ u_fine

    # Residual: (K - λM)u ≈ 0
    losses['residual'] = torch.mean((Ku - λ * Mu) ** 2)
    
    # Normalization: u^T M u = 1
    losses['normalization'] = ((u_fine.T @ Mu) - 1.0) ** 2

    # Orthogonality to previous eigenvectors
    losses['orthogonality'] = torch.tensor(0.0, device=u_fine.device, dtype=u_fine.dtype)
    if u_previous is not None and len(u_previous) > 0:
        for u_prev in u_previous:
            overlap = (u_fine.T @ (M_fine @ u_prev))
            losses['orthogonality'] += overlap ** 2

    # Smoothness regularization
    losses['smoothness'] = torch.mean((u_fine[1:] - u_fine[:-1]) ** 2)
    
    total = sum(weights[k] * losses[k] for k in losses)
    return total, losses

def adaptive_weights(epoch, losses_history=None):
    return {
        'residual': 1.0,
        'normalization': 10.0 * np.exp(-epoch / 100),
        'orthogonality': 5.0,
        'smoothness': 0.01
    }

# =========================
# Upscaler Training
# =========================
def train_upscaler(level_from, level_to, λ_coarse, u_coarse, K_fine, M_fine, idx_coarse, 
                   n_eigenpairs=10, n_epochs=1000, lr=1e-3, device=None):
    if device is None:
        device = get_device()
    
    n_coarse = u_coarse.shape[0]
    n_fine = K_fine.shape[0]

    print(f"Training upscaler: {n_coarse} → {n_fine} (level {level_from} → {level_to})")

    λ_fine = torch.zeros(n_eigenpairs)
    u_fine_all = []

    # Move matrices to GPU
    K_t = torch.tensor(K_fine, dtype=torch.float32, device=device)
    M_t = torch.tensor(M_fine, dtype=torch.float32, device=device)

    for i in range(n_eigenpairs):
        print(f"  Eigenpair {i+1}/{n_eigenpairs}")

        # Initialize eigenvalue from coarse solution
        λ_init = λ_coarse[i]
        
        # Convert coarse eigenvector to tensor and move to GPU
        u_c = torch.tensor(u_coarse[:, i], dtype=torch.float32, device=device)
        
        # Initialize eigenvalue as trainable parameter on GPU
        λ = torch.tensor(λ_init, dtype=torch.float32, device=device, requires_grad=True)

        # Create and move upscaler model to GPU
        model = HierarchicalUpscaler(n_coarse, n_fine).to(device)
        optimizer = torch.optim.Adam(list(model.parameters()) + [λ], lr=lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=50, factor=0.5)

        best_loss = float('inf')
        patience_counter = 0

        for epoch in range(n_epochs):
            # Upscale coarse eigenvector to fine mesh
            u_f = model(u_c)
            
            # Compute loss with adaptive weights
            weights = adaptive_weights(epoch)
            loss, loss_dict = compute_loss(u_f, λ, K_t, M_t, u_previous=u_fine_all, weights=weights)

            # Optimization step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step(loss)

            # Logging
            if epoch % 100 == 0:
                print(f"    Epoch {epoch}: Loss={loss.item():.2e}, λ={λ.item():.6f}, "
                      f"residual={loss_dict['residual'].item():.2e}")

            # Early stopping
            if loss.item() < best_loss:
                best_loss = loss.item()
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter > 200:
                    print(f"    Early stopping at epoch {epoch}")
                    break

        # Store results (move back to CPU for storage)
        λ_fine[i] = λ.detach().cpu()
        u_fine_all.append(u_f.detach())  # Keep on GPU for next iteration

    # Convert to numpy (move to CPU)
    u_fine_matrix = torch.stack(u_fine_all, dim=1).cpu().numpy()
    return λ_fine.numpy(), u_fine_matrix

# =========================
# Simple Rayleigh–Ritz Refinement
# =========================
def simple_rayleigh_ritz_refinement(λ_init, u_init, K, M, n_iter=3):
    """
    Refine eigenpairs using Rayleigh quotient iteration and Gram-Schmidt orthogonalization.
    """
    λ_refined = λ_init.copy()
    u_refined = u_init.copy()

    for it in range(n_iter):
        for i in range(len(λ_init)):
            u = u_refined[:, i]
            Ku = K @ u
            Mu = M @ u
            
            # Update eigenvalue using Rayleigh quotient
            λ_refined[i] = (u.T @ Ku) / (u.T @ Mu)

            # Orthogonalize against previous eigenvectors
            for j in range(i):
                overlap = u.T @ M @ u_refined[:, j]
                u -= overlap * u_refined[:, j]

            # Normalize
            norm = np.sqrt(u.T @ M @ u)
            if norm > 1e-10:
                u_refined[:, i] = u / norm
            else:
                print(f"Warning: Near-zero norm at iteration {it}, eigenvector {i}")

    return λ_refined, u_refined

# =========================
# Hierarchical Eigen Solve
# =========================
def hierarchical_eigensolve(K_full, M_full, n_eigenpairs=10, method='uniform', levels=None, device=None, 
                           skip_null_modes=0, regularization=1e-8):
    """
    Solve generalized eigenvalue problem K u = λ M u using hierarchical neural upscaling.
    
    Parameters:
    -----------
    skip_null_modes : int
        Number of near-zero eigenvalues to skip (e.g., rigid body modes)
    regularization : float
        Small value added to K diagonal for stability
    """
    if device is None:
        device = get_device()
    
    # Add small regularization to avoid numerical issues with null modes
    K_reg = K_full + regularization * M_full
    
    # Build hierarchy of coarse meshes
    K_h, M_h, idx_h = build_hierarchy(K_reg, M_full, levels=levels, method=method)
    level_keys = sorted(K_h.keys())

    # Solve coarsest level exactly (on CPU with scipy)
    print("\nSolving coarsest level...")
    K_coarse = K_h[level_keys[0]]
    M_coarse = M_h[level_keys[0]]
    
    # Request more modes to account for skipped ones
    n_modes_coarse = n_eigenpairs + skip_null_modes
    λ_all, u_all = eigh(K_coarse, M_coarse)
    
    # Skip null/near-zero modes
    if skip_null_modes > 0:
        print(f"Skipping first {skip_null_modes} modes (null/rigid body modes)")
        print(f"Skipped eigenvalues: {λ_all[:skip_null_modes]}")
        λ_current = λ_all[skip_null_modes:skip_null_modes + n_eigenpairs]
        u_current = u_all[:, skip_null_modes:skip_null_modes + n_eigenpairs]
    else:
        λ_current = λ_all[:n_eigenpairs]
        u_current = u_all[:, :n_eigenpairs]
    
    print(f"Using eigenvalues: {λ_current[:min(5, n_eigenpairs)]}")

    # Progressive refinement through hierarchy
    for i in range(1, len(level_keys)):
        level_from = level_keys[i-1]
        level_to = level_keys[i]

        print(f"\n--- Refining from level {level_from} to level {level_to} ---")
        
        # Neural upscaling (on GPU)
        λ_current, u_current = train_upscaler(
            level_from, level_to,
            λ_current, u_current,
            K_h[level_to], M_h[level_to],
            idx_coarse=idx_h[level_from],
            n_eigenpairs=n_eigenpairs,
            device=device
        )

        # Post-refinement with Rayleigh-Ritz (on CPU)
        print(f"  Applying Rayleigh-Ritz refinement...")
        λ_current, u_current = simple_rayleigh_ritz_refinement(
            λ_current, u_current,
            K_h[level_to], M_h[level_to],
            n_iter=3
        )
        
        print(f"  Refined eigenvalues: {λ_current[:min(5, n_eigenpairs)]}")
    
    # Remove regularization from final eigenvalues
    λ_current = λ_current - regularization

    return λ_current, u_current

# =========================
# Test Matrices
# =========================
def generate_test_matrices(n, matrix_type='laplacian'):
    """
    Generate test matrix pairs (K, M) for eigenvalue problems.
    """
    if matrix_type == 'laplacian':
        K = np.diag(2 * np.ones(n)) - np.diag(np.ones(n-1), 1) - np.diag(np.ones(n-1), -1)
        M = np.eye(n)
    elif matrix_type == 'tridiagonal':
        K = np.diag(2 + 0.1 * np.arange(n)) - np.diag(np.ones(n-1), 1) - np.diag(np.ones(n-1), -1)
        M = np.diag(1 + 0.01 * np.arange(n))
    elif matrix_type == 'random_spd':
        np.random.seed(42)
        A = np.random.randn(n, n)
        B = np.random.randn(n, n)
        K = A @ A.T + n * np.eye(n)
        M = B @ B.T + n * np.eye(n)
    else:
        raise ValueError(f"Unknown type: {matrix_type}")
    return K, M

# =========================
# Verification
# =========================
def verify_eigenpairs(λ, u, K, M, name=""):
    """
    Verify quality of computed eigenpairs.
    """
    print(f"\n{name} Verification:")
    n_eig = len(λ)
    for i in range(n_eig):
        res = np.linalg.norm(K @ u[:, i] - λ[i] * M @ u[:, i])
        norm = u[:, i].T @ M @ u[:, i]
        print(f"  λ[{i}]={λ[i]:.6f}, residual={res:.2e}, norm={norm:.6f}")


In [41]:
print("\n===== BUNNY TEST (n = 2503) =====")
from Mesh import Mesh

m = Mesh('bunny.obj')

centroid = m.verts.mean(0)
std_max = m.verts.std(0).max()

verts_new = (m.verts - centroid)/std_max

m = Mesh(verts = verts_new, connectivity = m.connectivity)

K, M = m.computeLaplacian()
print('Computing eigen values')
eigvals, eigvecs = eigh(K,M)

#levels = {0: 256, 1: 512, 2: 1024, 3: 2503}
levels = {0: 1024, 1: 2503}
λ_hier, u_hier = hierarchical_eigensolve(
    K, M,
    n_eigenpairs=10,
    levels=levels,
    skip_null_modes=1,      # Skip the ~0 eigenvalue
    regularization=1e-6,     # Small stabilizing shift
    method='uniform',        # or 'maxdist' for better sampling
    device=device
)
verify_eigenpairs(λ_hier, u_hier, K, M, "Bunny test")
print("✓ Bunny test done.")


===== BUNNY TEST (n = 2503) =====
Computing eigen values
Built level 0: 1024 DOFs
Built level 1: 2503 DOFs

Solving coarsest level...
Skipping first 1 modes (null/rigid body modes)
Skipped eigenvalues: [49.23205192]
Using eigenvalues: [50.75561299 51.81860999 54.6515201  54.8980322  57.45879678]

--- Refining from level 0 to level 1 ---
Training upscaler: 1024 → 2503 (level 0 → 1)
  Eigenpair 1/10
    Epoch 0: Loss=9.79e+00, λ=50.756611, residual=1.02e-02
    Epoch 100: Loss=3.25e-02, λ=50.793591, residual=3.14e-02
    Epoch 200: Loss=1.44e-02, λ=50.774284, residual=1.41e-02
    Epoch 300: Loss=9.75e-03, λ=50.731632, residual=9.39e-03
    Epoch 400: Loss=7.24e-03, λ=50.681938, residual=6.87e-03
    Epoch 500: Loss=5.57e-03, λ=50.632553, residual=5.16e-03
    Epoch 600: Loss=4.33e-03, λ=50.586979, residual=3.90e-03
    Epoch 700: Loss=3.36e-03, λ=50.546860, residual=2.91e-03
    Epoch 800: Loss=2.54e-03, λ=50.513058, residual=2.12e-03
    Epoch 900: Loss=1.86e-03, λ=50.485706, residual

In [42]:
import numpy as np
import torch
import torch.nn as nn
from scipy.linalg import eigh, solve

# =========================
# Device Configuration
# =========================
def get_device():
    """Get the best available device (CUDA if available, else CPU)"""
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device('cpu')
        print("Using CPU")
    return device

# =========================
# Hierarchy Construction
# =========================
def build_hierarchy(K_full, M_full, levels=None, method='uniform', preserve_boundary=True):
    """
    Build a hierarchy of downsampled matrices.
    For meshes with null eigenvalues, this can cause spectrum shift.
    """
    if levels is None:
        levels = {0: 32, 1: 256, 2: 2048, 3: K_full.shape[0]}

    n_full = K_full.shape[0]
    K_hierarchy, M_hierarchy, idx_hierarchy = {}, {}, {}

    for level, size in sorted(levels.items()):
        if size > n_full:
            size = n_full

        if method == 'uniform':
            idx = np.linspace(0, n_full - 1, size, dtype=int)
        elif method == 'random':
            np.random.seed(42 + level)
            idx = np.sort(np.random.choice(n_full, size, replace=False))
        elif method == 'maxdist':
            # Farthest point sampling for better coverage
            idx = farthest_point_sampling(K_full, size)
        else:
            raise ValueError(f"Unknown method: {method}")

        K_hierarchy[level] = K_full[np.ix_(idx, idx)]
        M_hierarchy[level] = M_full[np.ix_(idx, idx)]
        idx_hierarchy[level] = idx

        print(f"Built level {level}: {size} DOFs")

    return K_hierarchy, M_hierarchy, idx_hierarchy

def farthest_point_sampling(K, n_samples):
    """
    Select points using farthest point sampling (better for mesh downsampling).
    Uses the mass matrix structure as a proxy for distance.
    """
    n = K.shape[0]
    if n_samples >= n:
        return np.arange(n)
    
    indices = [0]
    distances = np.full(n, np.inf)
    
    for _ in range(n_samples - 1):
        last_idx = indices[-1]
        # Update distances (simple heuristic using row norms)
        new_dist = np.abs(K[last_idx, :])
        distances = np.minimum(distances, new_dist)
        distances[indices] = -np.inf
        indices.append(np.argmax(distances))
    
    return np.sort(np.array(indices))

# =========================
# Coarse → Fine Projection
# =========================
def project_coarse_to_fine(u_coarse, idx_coarse, n_fine, method='interpolate'):
    """
    Project coarse vector to fine mesh with interpolation.
    """
    u_fine = np.zeros(n_fine, dtype=float)
    u_fine[idx_coarse] = u_coarse
    
    if method == 'interpolate':
        # Linear interpolation between coarse points
        for i in range(len(idx_coarse) - 1):
            i_start = idx_coarse[i]
            i_end = idx_coarse[i + 1]
            if i_end - i_start > 1:
                # Interpolate values between coarse points
                u_fine[i_start:i_end+1] = np.linspace(u_coarse[i], u_coarse[i+1], i_end - i_start + 1)
    
    return u_fine

# =========================
# Neural Upscaler
# =========================
class HierarchicalUpscaler(nn.Module):
    def __init__(self, n_coarse, n_fine, hidden_factor=4):
        super().__init__()
        hidden = max(min(n_coarse * hidden_factor, n_fine // 2), 32)
        # Input: n_coarse, output: n_fine
        self.net = nn.Sequential(
            nn.Linear(n_coarse, hidden),
            nn.SiLU(),
            nn.Linear(hidden, hidden),
            nn.SiLU(),
            nn.Linear(hidden, n_fine)
        )

    def forward(self, u_coarse):
        """
        u_coarse: (n_coarse,) or (batch, n_coarse)
        Returns: (n_fine,) or (batch, n_fine)
        """
        if u_coarse.dim() == 1:
            return self.net(u_coarse.unsqueeze(0)).squeeze(0)
        else:
            return self.net(u_coarse)

# =========================
# Loss Function
# =========================
def compute_loss(u_fine, λ, K_fine, M_fine, u_previous=None, weights=None):
    if weights is None:
        weights = {'residual': 1.0, 'normalization': 10.0, 'orthogonality': 5.0, 'smoothness': 0.01}

    losses = {}
    Ku = K_fine @ u_fine
    Mu = M_fine @ u_fine

    # Residual: (K - λM)u ≈ 0
    losses['residual'] = torch.mean((Ku - λ * Mu) ** 2)
    
    # Normalization: u^T M u = 1
    losses['normalization'] = ((u_fine.T @ Mu) - 1.0) ** 2

    # Orthogonality to previous eigenvectors
    losses['orthogonality'] = torch.tensor(0.0, device=u_fine.device, dtype=u_fine.dtype)
    if u_previous is not None and len(u_previous) > 0:
        for u_prev in u_previous:
            overlap = (u_fine.T @ (M_fine @ u_prev))
            losses['orthogonality'] += overlap ** 2

    # Smoothness regularization
    losses['smoothness'] = torch.mean((u_fine[1:] - u_fine[:-1]) ** 2)
    
    total = sum(weights[k] * losses[k] for k in losses)
    return total, losses

def adaptive_weights(epoch, losses_history=None):
    return {
        'residual': 1.0,
        'normalization': 10.0 * np.exp(-epoch / 100),
        'orthogonality': 5.0,
        'smoothness': 0.01
    }

# =========================
# Upscaler Training
# =========================
def train_upscaler(level_from, level_to, λ_coarse, u_coarse, K_fine, M_fine, idx_coarse, 
                   n_eigenpairs=10, n_epochs=1000, lr=1e-3, device=None):
    if device is None:
        device = get_device()
    
    n_coarse = u_coarse.shape[0]
    n_fine = K_fine.shape[0]

    print(f"Training upscaler: {n_coarse} → {n_fine} (level {level_from} → {level_to})")

    λ_fine = torch.zeros(n_eigenpairs)
    u_fine_all = []

    # Move matrices to GPU
    K_t = torch.tensor(K_fine, dtype=torch.float32, device=device)
    M_t = torch.tensor(M_fine, dtype=torch.float32, device=device)

    for i in range(n_eigenpairs):
        print(f"  Eigenpair {i+1}/{n_eigenpairs}")

        # Initialize eigenvalue from coarse solution
        λ_init = λ_coarse[i]
        
        # Convert coarse eigenvector to tensor and move to GPU
        u_c = torch.tensor(u_coarse[:, i], dtype=torch.float32, device=device)
        
        # Initialize eigenvalue as trainable parameter on GPU
        λ = torch.tensor(λ_init, dtype=torch.float32, device=device, requires_grad=True)

        # Create and move upscaler model to GPU
        model = HierarchicalUpscaler(n_coarse, n_fine).to(device)
        optimizer = torch.optim.Adam(list(model.parameters()) + [λ], lr=lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=50, factor=0.5)

        best_loss = float('inf')
        patience_counter = 0

        for epoch in range(n_epochs):
            # Upscale coarse eigenvector to fine mesh
            u_f = model(u_c)
            
            # Compute loss with adaptive weights
            weights = adaptive_weights(epoch)
            loss, loss_dict = compute_loss(u_f, λ, K_t, M_t, u_previous=u_fine_all, weights=weights)

            # Optimization step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step(loss)

            # Logging
            if epoch % 100 == 0:
                print(f"    Epoch {epoch}: Loss={loss.item():.2e}, λ={λ.item():.6f}, "
                      f"residual={loss_dict['residual'].item():.2e}")

            # Early stopping
            if loss.item() < best_loss:
                best_loss = loss.item()
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter > 200:
                    print(f"    Early stopping at epoch {epoch}")
                    break

        # Store results (move back to CPU for storage)
        λ_fine[i] = λ.detach().cpu()
        u_fine_all.append(u_f.detach())  # Keep on GPU for next iteration

    # Convert to numpy (move to CPU)
    u_fine_matrix = torch.stack(u_fine_all, dim=1).cpu().numpy()
    return λ_fine.numpy(), u_fine_matrix

# =========================
# Simple Rayleigh–Ritz Refinement
# =========================
def simple_rayleigh_ritz_refinement(λ_init, u_init, K, M, n_iter=3):
    """
    Refine eigenpairs using Rayleigh quotient iteration and Gram-Schmidt orthogonalization.
    """
    λ_refined = λ_init.copy()
    u_refined = u_init.copy()

    for it in range(n_iter):
        for i in range(len(λ_init)):
            u = u_refined[:, i]
            Ku = K @ u
            Mu = M @ u
            
            # Update eigenvalue using Rayleigh quotient
            λ_refined[i] = (u.T @ Ku) / (u.T @ Mu)

            # Orthogonalize against previous eigenvectors
            for j in range(i):
                overlap = u.T @ M @ u_refined[:, j]
                u -= overlap * u_refined[:, j]

            # Normalize
            norm = np.sqrt(u.T @ M @ u)
            if norm > 1e-10:
                u_refined[:, i] = u / norm
            else:
                print(f"Warning: Near-zero norm at iteration {it}, eigenvector {i}")

    return λ_refined, u_refined

# =========================
# Hierarchical Eigen Solve
# =========================
def hierarchical_eigensolve(K_full, M_full, n_eigenpairs=10, method='uniform', levels=None, device=None, 
                           skip_null_modes=0, regularization=1e-8):
    """
    Solve generalized eigenvalue problem K u = λ M u using hierarchical neural upscaling.
    
    Parameters:
    -----------
    skip_null_modes : int
        Number of near-zero eigenvalues to skip (e.g., rigid body modes)
    regularization : float
        Small value added to K diagonal for stability
    """
    if device is None:
        device = get_device()
    
    # Add small regularization to avoid numerical issues with null modes
    K_reg = K_full + regularization * M_full
    
    # Build hierarchy of coarse meshes
    K_h, M_h, idx_h = build_hierarchy(K_reg, M_full, levels=levels, method=method)
    level_keys = sorted(K_h.keys())

    # Solve coarsest level exactly (on CPU with scipy)
    print("\nSolving coarsest level...")
    K_coarse = K_h[level_keys[0]]
    M_coarse = M_h[level_keys[0]]
    
    # Request more modes to account for skipped ones
    n_modes_coarse = n_eigenpairs + skip_null_modes
    λ_all, u_all = eigh(K_coarse, M_coarse)
    
    # Skip null/near-zero modes
    if skip_null_modes > 0:
        print(f"Skipping first {skip_null_modes} modes (null/rigid body modes)")
        print(f"Skipped eigenvalues: {λ_all[:skip_null_modes]}")
        λ_current = λ_all[skip_null_modes:skip_null_modes + n_eigenpairs]
        u_current = u_all[:, skip_null_modes:skip_null_modes + n_eigenpairs]
    else:
        λ_current = λ_all[:n_eigenpairs]
        u_current = u_all[:, :n_eigenpairs]
    
    print(f"Using eigenvalues: {λ_current[:min(5, n_eigenpairs)]}")

    # Progressive refinement through hierarchy
    for i in range(1, len(level_keys)):
        level_from = level_keys[i-1]
        level_to = level_keys[i]

        print(f"\n--- Refining from level {level_from} to level {level_to} ---")
        
        # Neural upscaling (on GPU)
        λ_current, u_current = train_upscaler(
            level_from, level_to,
            λ_current, u_current,
            K_h[level_to], M_h[level_to],
            idx_coarse=idx_h[level_from],
            n_eigenpairs=n_eigenpairs,
            device=device
        )

        # Post-refinement with Rayleigh-Ritz (on CPU)
        print(f"  Applying Rayleigh-Ritz refinement...")
        λ_current, u_current = simple_rayleigh_ritz_refinement(
            λ_current, u_current,
            K_h[level_to], M_h[level_to],
            n_iter=3
        )
        
        print(f"  Refined eigenvalues: {λ_current[:min(5, n_eigenpairs)]}")
    
    # Remove regularization from final eigenvalues
    λ_current = λ_current - regularization

    return λ_current, u_current

# =========================
# Test Matrices
# =========================
def generate_test_matrices(n, matrix_type='laplacian'):
    """
    Generate test matrix pairs (K, M) for eigenvalue problems.
    """
    if matrix_type == 'laplacian':
        K = np.diag(2 * np.ones(n)) - np.diag(np.ones(n-1), 1) - np.diag(np.ones(n-1), -1)
        M = np.eye(n)
    elif matrix_type == 'tridiagonal':
        K = np.diag(2 + 0.1 * np.arange(n)) - np.diag(np.ones(n-1), 1) - np.diag(np.ones(n-1), -1)
        M = np.diag(1 + 0.01 * np.arange(n))
    elif matrix_type == 'random_spd':
        np.random.seed(42)
        A = np.random.randn(n, n)
        B = np.random.randn(n, n)
        K = A @ A.T + n * np.eye(n)
        M = B @ B.T + n * np.eye(n)
    else:
        raise ValueError(f"Unknown type: {matrix_type}")
    return K, M

# =========================
# Verification
# =========================
def verify_eigenpairs(λ, u, K, M, name=""):
    """
    Verify quality of computed eigenpairs.
    """
    print(f"\n{name} Verification:")
    n_eig = len(λ)
    for i in range(n_eig):
        res = np.linalg.norm(K @ u[:, i] - λ[i] * M @ u[:, i])
        norm = u[:, i].T @ M @ u[:, i]
        print(f"  λ[{i}]={λ[i]:.6f}, residual={res:.2e}, norm={norm:.6f}")

def diagnose_hierarchy(K_full, M_full, levels=None, method='uniform', n_check=5):
    """
    Diagnose how downsampling affects eigenvalues.
    This helps understand if the hierarchy is appropriate.
    """
    print("\n" + "="*60)
    print("HIERARCHY DIAGNOSIS")
    print("="*60)
    
    K_h, M_h, idx_h = build_hierarchy(K_full, M_full, levels=levels, method=method)
    
    for level in sorted(K_h.keys()):
        K_level = K_h[level]
        M_level = M_h[level]
        
        # Compute first few eigenvalues
        λ_level, _ = eigh(K_level, M_level)
        λ_level = λ_level[:n_check]
        
        print(f"\nLevel {level} (size={K_level.shape[0]}):")
        print(f"  Eigenvalues: {λ_level}")
        
    # Compare first and last level
    λ_first = eigh(K_h[min(K_h.keys())], M_h[min(K_h.keys())])[0][:n_check]
    λ_last = eigh(K_h[max(K_h.keys())], M_h[max(K_h.keys())])[0][:n_check]
    
    print("\n" + "="*60)
    print("SPECTRUM SHIFT ANALYSIS:")
    print(f"Coarsest level: {λ_first}")
    print(f"Finest level:   {λ_last}")
    print(f"Ratio (should be ~1): {λ_first / λ_last}")
    print("="*60)
    
    if np.any(λ_first / λ_last > 10) or np.any(λ_first / λ_last < 0.1):
        print("\n⚠️  WARNING: Large spectrum shift detected!")
        print("   Downsampling dramatically changes eigenvalues.")
        print("   Consider:")
        print("   1. Using finer coarse meshes")
        print("   2. Using different downsampling strategy")
        print("   3. Using projection/interpolation operators")
    
    return K_h, M_h, idx_h


In [47]:
print("\n===== BUNNY TEST (n = 2503) =====")
from Mesh import Mesh

m = Mesh('bunny.obj')

centroid = m.verts.mean(0)
std_max = m.verts.std(0).max()

verts_new = (m.verts - centroid)/std_max

m = Mesh(verts = verts_new, connectivity = m.connectivity)

K, M = m.computeLaplacian()
print('Computing eigen values')
eigvals, eigvecs = eigh(K,M)

levels = {0: 128, 1: 512, 2: 1024, 3: 1500, 4: 2000, 5: 2503}
λ_hier, u_hier = hierarchical_eigensolve(
    K, M,
    n_eigenpairs=10,
    levels=levels,
    skip_null_modes=1,      # Skip the ~0 eigenvalue
    regularization=1e-6,     # Small stabilizing shift
    method='uniform',        # or 'maxdist' for better sampling
    device=device
)
verify_eigenpairs(λ_hier, u_hier, K, M, "Bunny test")
print("✓ Bunny test done.")


===== BUNNY TEST (n = 2503) =====
Computing eigen values
Built level 0: 128 DOFs
Built level 1: 512 DOFs
Built level 2: 1024 DOFs
Built level 3: 1500 DOFs
Built level 4: 2000 DOFs
Built level 5: 2503 DOFs

Solving coarsest level...
Skipping first 1 modes (null/rigid body modes)
Skipped eigenvalues: [141.70204874]
Using eigenvalues: [144.07828527 148.95798112 155.23216534 161.85716126 171.19768071]

--- Refining from level 0 to level 1 ---
Training upscaler: 128 → 512 (level 0 → 1)
  Eigenpair 1/10
    Epoch 0: Loss=9.76e+00, λ=144.079300, residual=2.59e-02
    Epoch 100: Loss=7.21e-02, λ=144.151138, residual=6.97e-02
    Epoch 200: Loss=2.18e-02, λ=144.174194, residual=1.99e-02
    Epoch 300: Loss=8.97e-03, λ=144.187927, residual=7.11e-03
    Epoch 400: Loss=5.63e-03, λ=144.199051, residual=3.85e-03
    Epoch 500: Loss=4.06e-03, λ=144.204391, residual=2.34e-03
    Epoch 600: Loss=3.12e-03, λ=144.203674, residual=1.48e-03
    Epoch 700: Loss=2.53e-03, λ=144.197464, residual=9.75e-04
  

In [46]:
λ_hier

array([41.510357, 40.49331 , 43.84867 , 45.69833 , 46.548016, 45.92576 ,
       50.122097, 51.026657, 51.08666 , 53.15215 ], dtype=float32)